<a href="https://colab.research.google.com/github/EmperoR1127/ml_project/blob/emperor/Sample_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [63]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
from scipy.io import arff
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)
# Where to save the figures
PROJECT_ROOT_DIR = "/content/drive/My Drive/"

#load the dataset
path = PROJECT_ROOT_DIR + "Data/H-1B_Disclosure_FY18_RAW_Data.csv"
df = pd.read_csv(path, encoding='utf-8')

#randomly select 20058 approved cases and 269 denied cases
df_approved = df[df['CASE_STATUS'] == "CERTIFIED"].sample(n = 20058, random_state = 42)
df_denied = df[df['CASE_STATUS'] != "CERTIFIED"].sample(n = 269, random_state = 42)
df = pd.concat([df_approved, df_denied], sort=False, axis=0).reset_index(drop=True)
df.to_csv(path_or_buf= PROJECT_ROOT_DIR + "ml_project_dataset/H-1B_Disclosure_RAW_Data.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
df.shape

(20327, 52)